# Analyse imagePoints

Our aim is to calibrate the cameras against the laser positions (3D) and projections of lasers into camrea images (as 2D image points where lines converge).

During this stage we want to:

1. Deselect any beamcaptures that are wrongly detected
2. Deselect any lasercaptures where the convergence point does not follow the circle in Figure 2 (NB : these should be re-selected after completing the camera calibration stage)
3. Evaluate residuals using Figure 3

In [283]:
import json
import numpy as np
import pandas as pd
import plotly.express as px

In [284]:
with open('../../Application/bin/data/Patch.json') as file:
	patch = json.load(file)
	

In [285]:
def find_node_by_name(node_name):
	for node_id in patch['Nodes']:
		if patch['Nodes'][node_id]['Name'] == node_name:
			return patch['Nodes'][node_id]
	raise(Exception("Node not found"))

In [286]:
calibrate_node = find_node_by_name('Calibrate')

In [287]:
first_camera = calibrate_node['Content']['cameraCaptures']['captures'][1]

In [288]:
#gather laser image points function
def gather_laser_image_points(camera_capture):
	laser_image_points = []
	laser_names = []
	for laser_capture in first_camera['laserCaptures']['captures']:
		if laser_capture['Selected']:
			laser_image_points.append(laser_capture['imagePointInCamera'])
			laser_names.append(laser_capture['laserAddress'])
	return laser_image_points, laser_names

In [289]:
laser_image_points, laser_names = gather_laser_image_points(first_camera)
data = np.array(laser_image_points)

# Figure 1

In [290]:
fig = px.scatter(x=data[:,0], y=data[:,1],hover_name=laser_names)
fig.show()

In [291]:
def get_laser_capture(camera_capture, laserAddress):
	for laser_capture in camera_capture['laserCaptures']['captures']:
		if int(laser_capture['laserAddress']) == laserAddress:
			return laser_capture

get_laser_capture(first_camera, 22).keys()

dict_keys(['Color', 'Selected', 'beamCaptures', 'date', 'directory', 'imagePointInCamera', 'laserAddress', 'linesWithCommonPointSolveResult', 'parameters', 'time', 'timestamp'])

In [292]:
# gather data
data_points = []
for camera_capture in calibrate_node['Content']['cameraCaptures']['captures']:
	for laser_capture in camera_capture['laserCaptures']['captures']:
		if laser_capture['Selected']:
			data_point = {}
			data_point['Camera'] = "{0} {1}".format(camera_capture['date'], camera_capture['time'])
			data_point['x'] = laser_capture['imagePointInCamera'][0]
			data_point['y'] = laser_capture['imagePointInCamera'][1]
			data_point['Laser'] = "Laser {0}".format(laser_capture['laserAddress'])
			
			if 'residual' in data_point:
				data_point['residual'] = laser_capture['laserAddress']
			else:
				data_point['residual'] = 0
				
			#count active beam captures
			active_beam_capture_count = 0
			for beam_capture in laser_capture['beamCaptures']['captures']:
				if beam_capture['Selected']:
					active_beam_capture_count = active_beam_capture_count + 1
			data_point['Active beam captures'] = active_beam_capture_count
			
			data_points.append(data_point)



# Figure 2

Disable LaserCaptures until we have good looking ellipses below:

Then we keep solving the intiial camera calibration, progressively removiving points with largest values until we get a good camera solve (e.g. values > 20k can be confusing for residuals)

In [293]:
fig = px.scatter(data_points
	, x='x'
	, y='y'
	, hover_name='Laser'
	, hover_data=['Active beam captures']
	, color='Camera')
fig.update_yaxes(autorange="reversed")
fig.show()

In [294]:
for data_point in data_points:
	if data_point['x'] == 0:
		print(data_point)

{'Camera': 'Thu 2022.05.19 22:16', 'x': 0.0, 'y': 0.0, 'Laser': 'Laser 1', 'residual': 0, 'Active beam captures': 9}
{'Camera': 'Thu 2022.05.19 22:16', 'x': 0.0, 'y': 0.0, 'Laser': 'Laser 2', 'residual': 0, 'Active beam captures': 9}


In [295]:
calibrate_node['Content']['cameraCaptures']['captures'][0]['date']

'Thu 2022.05.19'

# Figure 3

Two figures below show where most of the errors are.

Generally we use the `Prune/Beam captures by residual` function on the Calibrate node (e.g. with a value of 1000px).

In [296]:
# get residuals on beamCaptures

data_points = []
for camera_capture in calibrate_node['Content']['cameraCaptures']['captures']:
	for laser_capture in camera_capture['laserCaptures']['captures']:
		if laser_capture['Selected']:
			beam_capture_index = 0
			for beam_capture in laser_capture['beamCaptures']['captures']:
				if beam_capture['Selected']:
					data_point = {}
					data_point['Camera'] = "{0} {1}".format(camera_capture['date'], camera_capture['time'])
					data_point['Laser'] = laser_capture['laserAddress']
					data_point['projectionPoint_x'] = beam_capture['projectionPoint'][0]
					data_point['projectionPoint_y'] = beam_capture['projectionPoint'][1]
					data_point['residual'] = beam_capture['residual']
					data_point['beam_capture_index'] = beam_capture_index
					data_points.append(data_point)
					beam_capture_index += 1


fig = px.bar(data_points
	, x='Camera'
	, color='Laser'
	, y='residual'
	, hover_name='Laser'
	, hover_data=['beam_capture_index', 'projectionPoint_x', 'projectionPoint_y']
	, barmode="group")
fig.update_yaxes(autorange="reversed")
fig.show()

In [300]:
residuals = []
for data_point in data_points:
    residuals.append(data_point['residual'])
mean_residual = np.mean(residuals)

fig = px.scatter(data_points
    , x='residual'
    , y='Laser'
    , color='Camera'
    , hover_name='Laser'
	, hover_data=['beam_capture_index', 'projectionPoint_x', 'projectionPoint_y']
    , title="Mean residual {0} px".format(mean_residual))
fig.show()

# Notes from Greece

https://paper.dropbox.com/doc/KC68KC72-Calibration-log-v2--Bh4BhpiBl8CL5XP2Buzoq82GAg-zJiuoF5tDxpK6u18Sw23M#:uid=745312935561881238403951&h2=Athens-night-calibration-log

In [ ]:
# get all laser addresses
s = set()
for data_point in data_points:
	address = data_point['']
	s.add(data_point)